# CMSC320 Final Project
## Kinsey Smith, Sarah Bullard, Yiwen Shen

In [1]:
from IPython.display import HTML, display
display(HTML("<table><tr><td><img src='blm_twitter_image.jpg'></td><td><img src='Twitter_bird_logo_2012.svg.png'></td><td><img src='trump_twitter_image.jpg'></td></tr></table>"))

,,


<i> how to make these images less ugly? <i/>

### Introduction
Our project is surrounding the Twitter accounts of the Black Lives Matter Movement (@Blklivesmatter) and Donald Trump (@realDonaldTrump). We focused on the sentiment of the tweets of each account versus each individual tweet's replies. Our question was to find out what account would have more stark of a difference between tweets and their replies - in other words, whether positive tweets by BLM or Donald Trump would have more negative replies. <b> Our hypothesis was that Donald Trump's account would have more negative replies to his positive tweets. <--- Do you guys like this hypothesis? </b>

Sentiment analysis is a way of classifying a text as positive, negative, or neutral using text analysis. <i>FIX THAT SENTENCE</i>. However, because of the complex and sarcastic components of the English language, sentiment analysis is not a sole way of categorizing something as positive or negative. Because of this, we needed other factors to tell the sentiment of a reply. In order to do this, we created a feature vector and used that to classify the sentiment of the reply and used SVM machine learning in order to have the machine classify it for us. <i>FIX THAT SENTENCE</i>

The features in our feature vector are as follows. 

Our first feature was the original sentiment analysis, because although it is not reliably conclusive on its own, it can tell us something about the mood of the sentence. 

The second feature we worked on focused on the user who posted the reply to the specific tweet. We checked whether or not the user was following other accounts that aligned with Donald Trump's or BLM's views, including politicians of either party.

<b> Fix the two sentences above and continue adding features when we have them. Include issues we ran into and problems we faced/solutions we came up with as well? </b>


In [10]:
import tweepy
import textblob
import numpy as np
import pandas as pd
import collections
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

### Accessing Twitter's Data

In order to access Twitter's API, we had to create applications and personally get authentication tokens. Even though we are all allowed access to Twitter, we cannot give out these confidential tokens on this public notebook. In order to get past this hurdle, we created a function that would pull from our own files on our own machines for these tokens. In the cell below is a copy of credentials.py, without the confidential information. 

Once we did that, we started querying. However, since Twitter has a rule of a maximum of 15 queries in 15 minutes, and we were querying for replies of more than 15 tweets, we had to find another way to access the data when doing our project. We added the tweets of both accounts and their replies up to <b> INSERT DATE CSV WAS CREATED HERE </b> to csvs to work with them. We slowly added them to circumvent the 15-queries-maximum rule. See below the code we used to access Twitter's API in order to get the data. 

### Collecting the Data in a Functional Way

Now that the data is in a csv and collected, we can put it into a dataframe to work with. 

This dataframe will only include those tweets, as well as their replies, stemming from a year ago until now, as the most controversial times for both accounts are within that time frame. Specifically, Donald Trump became President of the United States in January and was controversial in the fall leading up to the election. Even though the Black Lives Matter movement was created in 2013 and became nationally recognized in 2014 after the shootings of Michael Brown and Eric Garner, it would be unrelated to our topic to compare the sentiment analysis of Donald Trump before he became controversial in the magnitude that he is now.

We also deleted unnecessary columns in the tweet json, like followers_count and following, which don't matter to what we are doing. 

In [18]:
data_donald = pd.read_csv("tweets_trump.csv")
del data_donald["username"]
del data_donald["followers_count"]
del data_donald["listed_count"]
del data_donald["following"]
del data_donald["favorites"]
del data_donald["verified"]
del data_donald["default_profile"]
del data_donald["statuses_count"]
del data_donald["description"]
del data_donald["geo_enabled"]
del data_donald["contributors_enabled"]
del data_donald["tweet_lat"]
del data_donald["tweet_long"]
del data_donald["tweet_source"]
del data_donald["tweet_in_reply_to_screen_name"]
del data_donald["tweet_direct_reply"]
del data_donald["tweet_retweet_count"]
del data_donald["tweet_favorite_count"]
del data_donald["tweet_hashtags_count"]
del data_donald["tweet_urls"]
del data_donald["tweet_urls_count"]
del data_donald["tweet_user_mentions_count"]
data_donald.head()

,name,location,time_zone,tweet_time,tweet_text,tweet_retweet_status,tweet_hashtags,tweet_user_mentions,tweet_media_type,tweet_contributors
0,Donald J. Trump,"Washington, DC",Eastern Time (US & Canada),2017-12-07 21:10:28,"Across the battlefields, oceans, and harrowing...",False,NaN,NaN,NaN,NaN
1,Donald J. Trump,"Washington, DC",Eastern Time (US & Canada),2017-12-07 20:52:49,"Today, as we Remember Pearl Harbor, it was an ...",False,NaN,NaN,NaN,NaN
2,Donald J. Trump,"Washington, DC",Eastern Time (US & Canada),2017-12-07 20:04:20,"Today, the U.S. flag flies at half-staff at th...",False,NaN,WhiteHouse,NaN,NaN
3,Donald J. Trump,"Washington, DC",Eastern Time (US & Canada),2017-12-07 16:16:19,"Today, our entire nation pauses to REMEMBER PE...",False,NaN,NaN,NaN,NaN
4,Donald J. Trump,"Washington, DC",Eastern Time (US & Canada),2017-12-07 15:04:54,"National Pearl Harbor Remembrance Day - ""A day...",False,NaN,NaN,NaN,NaN


In [19]:
data_blm = pd.read_csv("tweets_blm.csv")
del data_blm["username"]
del data_blm["followers_count"]
del data_blm["listed_count"]
del data_blm["following"]
del data_blm["favorites"]
del data_blm["verified"]
del data_blm["default_profile"]
del data_blm["statuses_count"]
del data_blm["description"]
del data_blm["geo_enabled"]
del data_blm["contributors_enabled"]
del data_blm["tweet_lat"]
del data_blm["tweet_long"]
del data_blm["tweet_source"]
del data_blm["tweet_in_reply_to_screen_name"]
del data_blm["tweet_direct_reply"]
del data_blm["tweet_retweet_count"]
del data_blm["tweet_favorite_count"]
del data_blm["tweet_hashtags_count"]
del data_blm["tweet_urls"]
del data_blm["tweet_urls_count"]
del data_blm["tweet_user_mentions_count"]
data_blm.head()

,name,location,time_zone,tweet_time,tweet_text,tweet_retweet_status,tweet_hashtags,tweet_user_mentions,tweet_media_type,tweet_contributors
0,Black Lives Matter,worldwide,NaN,2017-12-08 00:56:59,RT @KofiAdemola: #HandsOffJerusalemChi #FreePa...,True,"HandsOffJerusalemChi, FreePalestine","KofiAdemola, BLMChi, Blklivesmatter",NaN,NaN
1,Black Lives Matter,worldwide,NaN,2017-12-08 00:51:59,RT @KofiAdemola: HandsOffJerusalemChi #FreePal...,True,FreePalestine,"KofiAdemola, BLMChi, Blklivesmatter",NaN,NaN
2,Black Lives Matter,worldwide,NaN,2017-12-07 22:51:43,RT @BLMChi: Join us next Thursday!!! https://t...,True,NaN,BLMChi,photo,NaN
3,Black Lives Matter,worldwide,NaN,2017-12-07 22:46:42,RT @BLMLA: A MUST READ on @Blklivesmatter! Fea...,True,NaN,"BLMLA, Blklivesmatter, OsopePatrisse, DocMelly...",NaN,NaN
4,Black Lives Matter,worldwide,NaN,2017-12-07 22:41:41,"RT @DocMellyMel: One of the most in-depth, tho...",True,NaN,"DocMellyMel, Blklivesmatter",NaN,NaN


#### Sentiment Analysis

As explained before, sentiment analysis is a basic way of analyzing a text to tell if it has a positive, negative, or neutral sentiment. NLTK is the library we will use to do this section. This will be one of the features for the feature vector. 

## 662 words at this point. 

In [21]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [22]:
sia = SentimentIntensityAnalyzer()

In [25]:
sentiment_nltk_donald = []
for _,x in data_donald.iterrows():
    sentiment_nltk_donald.append((x["tweet_text"],sia.polarity_scores(x['tweet_text'])))
sentiment_nltk_blm = []
for _,x in data_blm.iterrows():
    sentiment_nltk_blm.append((x["tweet_text"],sia.polarity_scores(x['tweet_text'])))

In [27]:
i = 0
while i < 6:
    print(sentiment_nltk_donald[i])
    i = i + 1

('Across the battlefields, oceans, and harrowing skies of Europe and the Pacific throughout the war, one great battle... https://t.co/XelKHx4Ibo', {'neg': 0.224, 'neu': 0.618, 'pos': 0.158, 'compound': -0.1779})
('Today, as we Remember Pearl Harbor, it was an incredible honor to be joined with surviving Veterans of the attack o... https://t.co/c4ZYnXSAuj', {'neg': 0.113, 'neu': 0.691, 'pos': 0.196, 'compound': 0.3182})
('Today, the U.S. flag flies at half-staff at the @WhiteHouse, in honor of National Pearl Harbor Remembrance Day.... https://t.co/561NBdqSsb', {'neg': 0.0, 'neu': 0.849, 'pos': 0.151, 'compound': 0.4939})
('Today, our entire nation pauses to REMEMBER PEARL HARBOR--and the brave warriors who on that day stood tall and foug... https://t.co/xGiAt5hdnu', {'neg': 0.0, 'neu': 0.855, 'pos': 0.145, 'compound': 0.5267})
('National Pearl Harbor Remembrance Day - "A day that will live in infamy!" December 7, 1941', {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0})
('Go get the

## 30 nonredundant lines of code. 